# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

In [6]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')

import warnings
warnings.filterwarnings('ignore')

In [7]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [8]:
get_completion("what date is today")

"I'm sorry, I am an AI language model and I do not have real-time data. Please check the date on your device or search online for the current date."

## ConversationBufferMemory

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [19]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [22]:
conversation.predict(input="Hi, remember my horse is white")

'Hello! Yes, I remember that your horse is white. Is there anything specific you would like to know or talk about regarding your horse?'

In [16]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [21]:
conversation.predict(input="What is the color of my horse?")

'Your horse is white.'

In [23]:
print(memory.buffer)

Human: Hi, remember the my horse is white
AI: Hello! Yes, I remember that your horse is white. Is there anything specific you would like to know or talk about regarding your horse?
Human: What is the color of my horse?
AI: Your horse is white.
Human: Hi, remember my horse is white
AI: Hello! Yes, I remember that your horse is white. Is there anything specific you would like to know or talk about regarding your horse?


In [24]:
memory.load_memory_variables({})

{'history': 'Human: Hi, remember the my horse is white\nAI: Hello! Yes, I remember that your horse is white. Is there anything specific you would like to know or talk about regarding your horse?\nHuman: What is the color of my horse?\nAI: Your horse is white.\nHuman: Hi, remember my horse is white\nAI: Hello! Yes, I remember that your horse is white. Is there anything specific you would like to know or talk about regarding your horse?'}

In [25]:
memory = ConversationBufferMemory()

In [26]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [27]:
print(memory.buffer)

Human: Hi
AI: What's up


In [28]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [29]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [30]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [31]:
from langchain.memory import ConversationBufferWindowMemory

In [32]:
memory = ConversationBufferWindowMemory(k=1)               

In [33]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [34]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [35]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [36]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [37]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [38]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

## ConversationTokenBufferMemory

In [39]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [41]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [42]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [43]:
from langchain.memory import ConversationSummaryBufferMemory


In [44]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [45]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [46]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [47]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of our AI technology and how it can be applied to various industries and use cases.'

In [48]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, the human can demonstrate the capabilities of their AI technology and how it can be applied to various industries and use cases.'}